In [1]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error


**`1. Algorithm Development `**

In [ ]:
# stocks
tickers = ["AAPL", "NVDA", "TSLA"]

# 2 year window for data, 1d interval provides daily price points
data = yf.download(tickers, start="2024-01-01", end="2026-02-04", interval="1d")


apple_data = data['Close'][['AAPL']].rename(columns={'AAPL': 'Close'})
print(apple_data.head())

/tmp/ipython-input-1382162224.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2024-01-01", end="2026-02-04", interval="1d")
[*********************100%***********************]  3 of 3 completed

Ticker           Close
Date                  
2024-01-02  183.903214
2024-01-03  182.526260
2024-01-04  180.208130
2024-01-05  179.484955
2024-01-08  183.823959


Implement a simple Moving Average algorithm on the collected stock price data to kick-start your implementation strategy.

In [ ]:
import pandas as pd

def generate_signals(data, short_window=20, long_window=50):
    """
    Generates buy/sell signals based on SMA Crossover
    """
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']

    # calulate moving averages
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

    # 1.0 when short is above long, else 0.0
    signals['signal'] = 0.0
    signals['signal'][short_window:] = np.where(
        signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0
    )


    # 1.0 = Buy, -1.0 = Sell
    signals['positions'] = signals['signal'].diff()

    return signals

In [ ]:
generate_signals(apple_data)

/tmp/ipython-input-2336824961.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'][short_window:] = np.where(


,price,short_mavg,long_mavg,signal,positions
Date,,,,,
2024-01-02,183.903214,183.903214,183.903214,0.0,NaN
2024-01-03,182.526260,183.214737,183.214737,0.0,0.0
2024-01-04,180.208130,182.212535,182.212535,0.0,0.0
2024-01-05,179.484955,181.530640,181.530640,0.0,0.0
2024-01-08,183.823959,181.989304,181.989304,0.0,0.0
...,...,...,...,...,...
2026-01-28,256.440002,259.008496,268.737398,0.0,0.0
2026-01-29,258.279999,258.268497,268.454798,0.0,0.0
2026-01-30,259.480011,257.649498,268.295198,0.0,0.0


In [ ]:
simple_ma = generate_signals(apple_data)
valid_simple = simple_ma.dropna()

simple_mae = mean_absolute_error(valid_simple['price'], valid_simple['short_mavg'])
simple_rmse = np.sqrt(mean_squared_error(valid_simple['price'], valid_simple['short_mavg']))

print(f"Simple Moving Average MAE: {simple_mae}")
print(f"Simple Moving Average RMSE: {simple_rmse}")

Simple Moving Average MAE: 6.789572460730005
Simple Moving Average RMSE: 8.593344744354912


/tmp/ipython-input-2336824961.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'][short_window:] = np.where(


Discuss more advanced algorithms: ARIMA, LSTM, etc. Explore evaluation metrics for time series data: Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), etc.


In [ ]:

#  ARIMA time series model to forecast the next day's price
def generate_arima_signals(data, order=(5,1,0)):

    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']

    forecasts = []
    history = list(data['Close'].values)

    model = ARIMA(history, order=order)
    model_fit = model.fit()

    # generating predictions for the entire dataset
    signals['predicted_price'] = model_fit.fittedvalues

    # buy if the predicted price for tomorrow is higher than today's actual price
    signals['signal'] = 0.0
    signals['signal'] = np.where(signals['predicted_price'] > signals['price'], 1.0, 0.0)

    signals['positions'] = signals['signal'].diff()

    return signals, model_fit

# run the advanced ARIMA model
arima_signals, arima_model = generate_arima_signals(apple_data)

# eval metrics for ARIMA
arima_mae = mean_absolute_error(arima_signals['price'], arima_signals['predicted_price'])
arima_rmse = np.sqrt(mean_squared_error(arima_signals['price'], arima_signals['predicted_price']))

print(f"ARIMA Mean Absolute Error (MAE): {arima_mae}")
print(f"ARIMA Root Mean Squared Error (RMSE): {arima_rmse}")

ARIMA Mean Absolute Error (MAE): 2.8051729806914203
ARIMA Root Mean Squared Error (RMSE): 8.811924685030654


Select an algorithm or develop a hybrid algorithm to predict the stock prices and provide buy/sell
signals.
Implement the chosen algorithm(s) in Python. Create functions that generate buy/sell signals based
on the stock price data.

In [ ]:
import pandas as pd
import numpy as np

# moving average to find the trend (short or long) and rsi to find the entry point
def generate_hybrid_signals(data, short_window=20, long_window=50, rsi_period=14):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']

    # moving averages
    signals['short_mavg'] = data['Close'].rolling(window=short_window).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window).mean()

    # rsi calculations
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / loss
    signals['rsi'] = 100 - (100 / (1 + rs))

    # hybrid logic
    signals['signal'] = 0.0
    signals['signal'] = np.where(
        (signals['short_mavg'] > signals['long_mavg']) & (signals['rsi'] < 70), 1.0, 0.0
    )

    signals['positions'] = signals['signal'].diff()
    return signals

In [ ]:
signals = generate_hybrid_signals(apple_data)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# remove NaNs
hybrid_signals = generate_hybrid_signals(apple_data)
valid_hybrid = hybrid_signals.dropna()

# eval metrics for the hybrid model
hybrid_mae = mean_absolute_error(valid_hybrid['price'], valid_hybrid['short_mavg'])
hybrid_rmse = np.sqrt(mean_squared_error(valid_hybrid['price'], valid_hybrid['short_mavg']))

print(f"Hybrid Model Mean Absolute Error (MAE): {hybrid_mae}")
print(f"Hybrid Model Root Mean Squared Error (RMSE): {hybrid_rmse}")

Hybrid Model Mean Absolute Error (MAE): 7.056911703009354
Hybrid Model Root Mean Squared Error (RMSE): 8.859934231238464


In [ ]:
#LSTM Method - An LSTM NN was used to predict future stock prices,
#and moving average logic was applied to the predicted prices to generate buy and sell signals

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

#historical data to train model, convert and scale
def prepare_lstm_data(prices, lookback=60):
    prices = np.array(prices).reshape(-1, 1)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(prices)

    X, y = [], []
    for i in range(lookback, len(scaled)):
        X.append(scaled[i - lookback:i, 0])
        y.append(scaled[i, 0])

    X = np.array(X)
    y = np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    return X, y, scaler

#build lstm model
def train_lstm(prices, lookback=60, epochs=10):
    X, y, scaler = prepare_lstm_data(prices, lookback)

    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], 1)))
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")
    model.fit(X, y, epochs=epochs, batch_size=32, verbose=0)

    return model, scaler

#generates price predictions + align with date
def predict_lstm(model, prices, scaler, lookback=60):
    prices = np.array(prices).reshape(-1, 1)
    scaled = scaler.transform(prices)

    X = []
    for i in range(lookback, len(scaled)):
        X.append(scaled[i - lookback:i, 0])

    X = np.array(X).reshape((-1, lookback, 1))
    preds = model.predict(X, verbose=0)
    preds = scaler.inverse_transform(preds)

    preds = np.concatenate((np.full((lookback, 1), np.nan), preds))
    return pd.Series(preds.flatten(), index=apple_data.index)


In [ ]:
#smooth prices to identify any trends
def moving_avg(prices, window):
    return prices.rolling(window=window).mean()

#turns the prices into buy sell or hold signals
def mov_avg_alg(prices, short_window=10, long_window=30):
    signals = pd.DataFrame(index=prices.index)
    signals['price'] = prices

    signals['short_ma'] = moving_avg(prices, short_window)
    signals['long_ma'] = moving_avg(prices, long_window)

    signals['signal'] = 0.0
    signals['signal'] = np.where(
        signals['short_ma'] > signals['long_ma'], 1.0, 0.0
    )

    signals['positions'] = signals['signal'].diff()
    return signals



In [ ]:

ticker = "AAPL"
data = yf.download(ticker, start="2024-01-01", end="2026-02-04", interval="1d")

apple_data = data[['Close']].dropna()
prices = apple_data['Close']

# train LSTM
model, scaler = train_lstm(prices, lookback=60, epochs=10)

# predict prices
predicted_prices = predict_lstm(model, prices, scaler)

# generate MA signals from LSTM predictions
signals = mov_avg_alg(predicted_prices)

# remove NaNs
valid = signals.dropna()


/tmp/ipython-input-3266106347.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2026-02-04", interval="1d")
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:

# evaluation metrics, compares after 60 days due to prediction
actual_price = prices.squeeze()
predicted_price = predicted_prices.squeeze()

lstm_df = pd.concat(
    [actual_price, predicted_price],
    axis=1
).dropna()

lstm_df.columns = ['actual_price', 'predicted_price']

#compare predicted and real prices
lstm_mae = mean_absolute_error(
    lstm_df['actual_price'],
    lstm_df['predicted_price']
)

lstm_rmse = np.sqrt(mean_squared_error(
    lstm_df['actual_price'],
    lstm_df['predicted_price']
))

print(f"LSTM Price Prediction MAE: {lstm_mae}")
print(f"LSTM Price Prediction RMSE: {lstm_rmse}")

LSTM Price Prediction MAE: 5.761430641700482
LSTM Price Prediction RMSE: 7.471479305469179


**Concluding thoughts:**

ARIMA achieved the lowest MAE, indicating strong average predictive accuracy, while the LSTM model achieved the lowest RMSE, suggesting better robustness to large prediction errors. The hybrid MA + RSI strategy performed worse in terms of prediction error, as it is primarily designed for trend-based trading rather than price forecasting.

**`2. Mock Trading Environment `**

In [ ]:
# define initial investment fund of $10,000
# track the portfolio value over time based on the buy/sell actions
def run_mock_trading(signals_df, initial_fund=10000.0):

    portfolio = pd.DataFrame(index=signals_df.index)
    portfolio['price'] = signals_df['price']
    portfolio['cash'] = initial_fund
    portfolio['shares'] = 0.0
    portfolio['total_value'] = initial_fund

    current_cash = initial_fund
    current_shares = 0

    # loop through signals to simulate trades
    for i in range(len(signals_df)):
        price = signals_df['price'].iloc[i]
        signal_action = signals_df['positions'].iloc[i]

        if signal_action == 1.0: # buy
            shares_to_buy = current_cash // price # Buy as many as possible
            current_shares += shares_to_buy
            current_cash -= (shares_to_buy * price)

        elif signal_action == -1.0: # sell
            current_cash += (current_shares * price)
            current_shares = 0

        # update records for each day
        portfolio.iloc[i, portfolio.columns.get_loc('cash')] = current_cash
        portfolio.iloc[i, portfolio.columns.get_loc('shares')] = current_shares
        portfolio.iloc[i, portfolio.columns.get_loc('total_value')] = current_cash + (current_shares * price)

    # calculate metrics
    # portfolio value
    portfolio['daily_returns'] = portfolio['total_value'].pct_change()

    total_return = (portfolio['total_value'].iloc[-1] / initial_fund) - 1
    # annualized return (assuming 252 trading days)
    ann_return = ((1 + total_return) ** (252 / len(portfolio))) - 1

    # Sharpe ratio
    sharpe = (portfolio['daily_returns'].mean() / portfolio['daily_returns'].std()) * np.sqrt(252)

    return portfolio, total_return, ann_return, sharpe

In [ ]:
lstm_signal_df = pd.DataFrame(index=prices.index)
lstm_signal_df['price'] = prices
lstm_signal_df['predicted_price'] = predicted_prices.squeeze()

# drop rows without predictions (first 60 days)
lstm_signal_df = lstm_signal_df.dropna()

# apply moving average to LSTM
lstm_signal_df['short_ma'] = lstm_signal_df['predicted_price'].rolling(window=10).mean()
lstm_signal_df['long_ma'] = lstm_signal_df['predicted_price'].rolling(window=30).mean()

# generate signals
lstm_signal_df['signal'] = 0.0
lstm_signal_df['signal'] = np.where(
    lstm_signal_df['short_ma'] > lstm_signal_df['long_ma'], 1.0, 0.0
)

# positions: +1 buy, -1 sell
lstm_signal_df['positions'] = lstm_signal_df['signal'].diff()

# drop remaining NaNs from MA windows
lstm_signal_df = lstm_signal_df.dropna()

In [ ]:
# Execute for Moving Average
portfolio_ma, total_ret_ma, ann_ret_ma, sharpe_val_ma = run_mock_trading(valid_simple)

print(f"Moving Average: Final Value: ${portfolio_ma['total_value'].iloc[-1]:.2f}")
print(f"Moving Average: Annualized Return: {ann_ret_ma:.2%}")
print(f"Moving Average: Sharpe Ratio: {sharpe_val_ma:.2f}")

# Execute for ARIMA Model
portfolio_arima, total_ret_arima, ann_ret_arima, sharpe_val_arima = run_mock_trading(arima_signals)

print(f"ARIMA Model: Final Value: ${portfolio_arima['total_value'].iloc[-1]:.2f}")
print(f"ARIMA Model: Annualized Return: {ann_ret_arima:.2%}")
print(f"ARIMA Model: Sharpe Ratio: {sharpe_val_arima:.2f}")

# Execute for Hybrid Model
portfolio_hybrid, total_ret_hybrid, ann_ret_hybrid, sharpe_val_hybrid = run_mock_trading(valid_hybrid)

print(f"Hybrid Model: Final Value: ${portfolio_hybrid['total_value'].iloc[-1]:.2f}")
print(f"Hybrid Model: Annualized Return: {ann_ret_hybrid:.2%}")
print(f"Hybrid Model: Sharpe Ratio: {sharpe_val_hybrid:.2f}")

#Execute for LSTM Model
portfolio_lstm, total_ret_lstm, ann_ret_lstm, sharpe_val_lstm = run_mock_trading(lstm_signal_df)

print(f"LSTM + MA Model: Final Value: ${portfolio_lstm['total_value'].iloc[-1]:.2f}")
print(f"LSTM + MA Model: Annualized Return: {ann_ret_lstm:.2%}")
print(f"LSTM + MA Model: Sharpe Ratio: {sharpe_val_lstm:.2f}")

Moving Average: Final Value: $12802.70
Moving Average: Annualized Return: 12.64%
Moving Average: Sharpe Ratio: 0.76
ARIMA Model: Final Value: $9080.73
ARIMA Model: Annualized Return: -4.53%
ARIMA Model: Sharpe Ratio: -0.11
Hybrid Model: Final Value: $12498.49
Hybrid Model: Annualized Return: 12.56%
Hybrid Model: Sharpe Ratio: 0.77
LSTM + MA Model: Final Value: $11243.00
LSTM + MA Model: Annualized Return: 7.02%
LSTM + MA Model: Sharpe Ratio: 0.45


**Findings**

While the Simple Moving Average technically made about $300 more in total profit, the Hybrid Model is the best model because it has the highest Sharpe Ratio (0.77). A higher Sharpe ratio means the Hybrid model earned its profit with less volatility. By using the RSI filter, it likely avoided some risky trades that the Simple Moving Average took. In professional trading, a higher Sharpe ratio is usually preferred over slightly higher total returns because it suggests the strategy is more stable over the long term; thus, the Hybrid model performs best.

Maintain a record of the number of shares held for each stock and update it after each trade.

In [ ]:
# display preview of when each model bought and sold shares (show when a trade happens)
print("\n Shares Record for Moving Average (First 10 Trades)")
print(portfolio_ma[portfolio_ma['shares'].diff() != 0][['price', 'shares', 'cash', 'total_value']].head(10))

print("\n Shares Record for ARIMA (First 10 Trades)")
print(portfolio_arima[portfolio_arima['shares'].diff() != 0][['price', 'shares', 'cash', 'total_value']].head(10))

print("\n Shares Record for Hybrid (First 10 Trades)")
print(portfolio_hybrid[portfolio_hybrid['shares'].diff() != 0][['price', 'shares', 'cash', 'total_value']].head(10))

print("\n Shares Record for LSTM + MA (First 10 Trades)")
print(portfolio_lstm[portfolio_lstm['shares'].diff() != 0][['price', 'shares', 'cash', 'total_value']].head(10))


 Shares Record for Moving Average (First 10 Trades)
                 price  shares          cash   total_value
Date                                                      
2024-01-03  182.526260     0.0  10000.000000  10000.000000
2024-01-31  182.674805    54.0    135.560547  10000.000000
2024-02-23  181.043137     0.0   9911.889923   9911.889923
2024-05-08  181.261337    54.0    123.777710   9911.889923
2024-08-20  225.242355     0.0  12286.864899  12286.864899
2024-08-27  226.753845    54.0     42.157257  12286.864899
2024-11-19  227.252197     0.0  12313.775909  12313.775909
2024-12-03  241.557495    50.0    235.901154  12313.775909
2025-01-27  228.825089     0.0  11677.155579  11677.155579
2025-03-06  234.528046    49.0    185.281342  11677.155579

 Shares Record for ARIMA (First 10 Trades)
                 price  shares          cash   total_value
Date                                                      
2024-01-02  183.903214     0.0  10000.000000  10000.000000
2024-01-03  182.52